In [44]:
import datetime
from dateutil.parser import parse
import itertools
from itertools import product
import math
import matplotlib.pyplot as plt
from multiprocessing.dummy import Pool as ThreadPool
import nolds
import numpy as np
import os
import pandas as pd
import pathlib
from PreprocessFcns import *
import pywt
import random
import scipy
from scipy.signal import butter, welch, filtfilt, resample
from scipy.stats import skew, kurtosis, entropy, pearsonr
import seaborn as sns
import sklearn
from sklearn import multiclass
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneGroupOut
import time
%matplotlib inline

In [45]:
path = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Watch DTW Analysis'
altdest = 'C:\\Users\\rpieper\\Desktop\\Parkinson\'s'

In [46]:
VisitNumber = {
    '2 Weeks: Time 0'   : 0,
    '2 Weeks: Time 30'  : 1,
    '2 Weeks: Time 60'  : 2,
    '2 Weeks: Time 90'  : 3,
    '2 Weeks: Time 120' : 4,
    '2 Weeks: Time 150' : 5,
    '1 Month'           : 6
}

ClinicTasks = {
    'Stndg'    : 'Standing',
    'Wlkg'     : 'Walking',
    'WlkgCnt'  : 'Walking while counting',
    'FtnR'     : 'Finger to nose--right hand',
    'FtnL'     : 'Finger to nose--left hand',
    'RamR'     : 'Alternating right hand movements',
    'RamL'     : 'Alternating left hand movements',
    'SitStand' : 'Sit to stand',
    'Drwg'     : 'Drawing on a paper',
    'Typg'     : 'Typing on a computer keyboard',
    'NtsBts'   : 'Assembling nuts and bolts',
    'Drnkg'    : 'Taking a glass of water and drinking',
    'Sheets'   : 'Organizing sheets in a folder',
    'Fldg'     : 'Folding towels',
    'Sitng'    : 'Sitting'
}

In [47]:
ClinicMeta = pd.read_csv(os.path.join(path, 'Metadata Tables', 'Table4.csv'), 
                         parse_dates = ['Start Timestamp (UTC)', 'Stop Timestamp (UTC)'])
ClinicMeta.head()

,SubjID,Visit,Task,TaskAbb,Start Timestamp (UTC),Stop Timestamp (UTC),Tremor - Left,Tremor - Right,Bradykinesia - Left,Bradykinesia - Right,Dyskinesia - Left,Dyskinesia - Right,Overall,Validated,Side
0,1003.0,2 Weeks: Time 0,Taking a glass of water and drinking,Drnkg,2017-07-03 15:52:57.090,2017-07-03 15:53:32.788,1.0,1.0,1.0,1.0,0.0,0.0,1.0,True,Left
1,1003.0,2 Weeks: Time 0,Drawing on a paper,Drwg,2017-07-03 15:47:43.937,2017-07-03 15:47:59.768,1.0,0.0,0.0,2.0,0.0,0.0,1.0,True,Left
2,1003.0,2 Weeks: Time 0,Folding towels,Fldg,2017-07-03 15:56:49.213,2017-07-03 15:57:38.648,1.0,1.0,1.0,1.0,0.0,0.0,1.0,True,Left
3,1003.0,2 Weeks: Time 0,Finger to nose--left hand,FtnL,2017-07-03 15:41:52.683,2017-07-03 15:42:22.277,1.0,0.0,1.0,NaN,0.0,0.0,1.0,True,Left
4,1003.0,2 Weeks: Time 0,Finger to nose--right hand,FtnR,2017-07-03 15:41:11.945,2017-07-03 15:41:43.837,0.0,1.0,NaN,1.0,0.0,0.0,1.0,True,Left


In [48]:
ClinicMetaRed = pd.DataFrame()
ClinicTask = pd.DataFrame()
for task in ClinicMeta.iterrows():
    if type(task[1]['Side']) == float:
        continue
    ClinicTask['SubjID'] = [int(task[1]['SubjID'])]
    ClinicTask['Visit'] = [VisitNumber[task[1]['Visit']]]
    ClinicTask['TaskAbb'] = [task[1]['TaskAbb']]
    ClinicTask['Tremor'] = [task[1]['Tremor - ' + task[1]['Side']]]
    ClinicTask['Bradykinesia'] = [task[1]['Bradykinesia - ' + task[1]['Side']]]
    ClinicTask['Dyskinesia'] = [task[1]['Dyskinesia - ' + task[1]['Side']]]
    ClinicTask['Overall'] = [task[1]['Overall']]
    ClinicTask['AccFile'] = [str(int(task[1]['SubjID'])) + '_' + 
                             str(VisitNumber[task[1]['Visit']]) + '_' + 
                             task[1]['TaskAbb'] + '.csv']
    if ClinicMetaRed.empty:
        ClinicMetaRed = ClinicTask
        continue
    ClinicMetaRed = pd.concat([ClinicMetaRed, ClinicTask], ignore_index = True)
ClinicMetaRed.head()

,SubjID,Visit,TaskAbb,Tremor,Bradykinesia,Dyskinesia,Overall,AccFile
0,1003,0,Drwg,1.0,0.0,0.0,1.0,1003_0_Drwg.csv
1,1003,0,Drwg,1.0,0.0,0.0,1.0,1003_0_Drwg.csv
2,1003,0,Fldg,1.0,1.0,0.0,1.0,1003_0_Fldg.csv
3,1003,0,FtnL,1.0,1.0,0.0,1.0,1003_0_FtnL.csv
4,1003,0,FtnR,0.0,NaN,0.0,1.0,1003_0_FtnR.csv


In [49]:
WalkTempMeta = ClinicMetaRed[(ClinicMetaRed.TaskAbb == 'Wlkg') & 
                             (ClinicMetaRed.Tremor > float(0))]
print(len(WalkTempMeta))
WalkTempMeta.head()

34


,SubjID,Visit,TaskAbb,Tremor,Bradykinesia,Dyskinesia,Overall,AccFile
88,1003,5,Wlkg,1.0,0.0,0.0,0.0,1003_5_Wlkg.csv
388,1009,0,Wlkg,2.0,1.0,0.0,1.0,1009_0_Wlkg.csv
403,1009,1,Wlkg,2.0,1.0,0.0,0.0,1009_1_Wlkg.csv
418,1009,2,Wlkg,2.0,1.0,0.0,0.0,1009_2_Wlkg.csv
433,1009,3,Wlkg,2.0,1.0,0.0,1.0,1009_3_Wlkg.csv


In [50]:
WalkTempFiles = {}
for sub in WalkTempMeta.SubjID.unique():
    subLab = 's' + str(sub)
    WalkTempFiles[subLab] = {}
    SubWalkTempFiles = WalkTempMeta[WalkTempMeta.SubjID == sub]
    for visit in SubWalkTempFiles.iterrows():
        if os.path.isfile(os.path.join(path, 'TaskAcc', visit[1]['AccFile'])):
            VisitFile = (pd.read_csv(os.path.join(path, 'TaskAcc', visit[1]['AccFile']), 
                                     parse_dates = ['timestamp']) 
                         [['timestamp', 'x', 'y', 'z']])
            VisitFile['timestamp'] = (pd.to_datetime(VisitFile['timestamp']).astype(np.int64) / 10**6).astype(np.int64)
            VisitFile.columns = ['Timestamp', 'X', 'Y', 'Z']
            visLab = 'v' + str(visit[1]['Visit'])
            WalkTempFiles[subLab][visLab] = VisitFile.values
    if not any(WalkTempFiles[subLab]):
        del WalkTempFiles[subLab]

In [51]:
WalkTempFiles.keys()

dict_keys(['s1003', 's1009', 's1020', 's1029', 's1043', 's1046', 's1048', 's1049', 's1050'])

In [52]:
scipy.io.savemat(os.path.join(dest, 'TremWlkg'), WalkTempFiles)

In [117]:
VisitFile.values

array([[ 1.50955305e+12,  9.78759770e-01,  5.92529300e-01,
         2.19726560e-02],
       [ 1.50955305e+12,  9.68994140e-01,  5.64941400e-01,
         6.07910160e-02],
       [ 1.50955305e+12,  9.32617200e-01,  5.43212900e-01,
         4.17480470e-02],
       ...,
       [ 1.50955309e+12,  9.38720700e-01,  3.90136720e-01,
        -4.25781250e-01],
       [ 1.50955309e+12,  9.33593750e-01,  3.98437500e-01,
        -4.28222660e-01],
       [ 1.50955309e+12,  9.49462900e-01,  3.72070300e-01,
        -4.29931640e-01]])